## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [109]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
from datetime import date

np.random.seed(0)

In [110]:
# Load up store_income_data.csv
df = pd.read_csv('store_income_data_task.csv')
# df.country.dropna()

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [111]:
def unique_countries():
    countries = df['country'].unique()
    print(f"There are {len(countries)} unique countries:", (countries))
    
df['country'] = df['country'].str.lower()
df['country'] = df['country'].str.strip()
df['country'] = df['country'].str.replace('.', '')
df['country'] = df['country'].str.strip('/')
unique_countries()

There are 11 unique countries: ['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' nan
 'england' '' 'united states of america' 's africasouth africa']


In [112]:
# There is an empty country value, it will be changed to 'other'
df['country'] = df['country'].fillna('other')

unique_countries()

There are 11 unique countries: ['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' 'other'
 'england' '' 'united states of america' 's africasouth africa']


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [113]:
matches = fuzzywuzzy.process.extract("uk", countries, limit=10, 
                                     scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# Inspect matches
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    strings = df[column].unique()
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match
    
    print("Task complete")

replace_matches_in_column(df=df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=df, column='country', string_to_match="britain")
replace_matches_in_column(df=df, column='country', string_to_match="united states")
replace_matches_in_column(df=df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=df, column='country', string_to_match="south africa")
replace_matches_in_column(df=df, column='country', string_to_match="uk")

# There are still several variations of country names to take in to account
df.country.replace('united kingdom','uk', inplace=True)
df.country.replace('britain','uk', inplace=True)
df.country.replace('england','uk', inplace=True)
df.country.replace('sa','south africa', inplace=True)
df.country.replace('s africasouth africa','south africa', inplace=True)
df.country.replace('america','united states', inplace=True)
df.country.replace('united states of america','united states', inplace=True)

unique_countries()

Task complete
Task complete
Task complete
Task complete
Task complete
Task complete
There are 5 unique countries: ['united states' 'uk' 'south africa' 'other' '']


C:\Users\scien\AppData\Local\Temp\ipykernel_26620\2568704524.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.country.replace('united kingdom','uk', inplace=True)
C:\Users\scien\AppData\Local\Temp\ipykernel_26620\2568704524.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [115]:
# df.head()

df['formatted_date'] = pd.to_datetime(df['date_measured'], format = '%d-%m-%Y')
df['days_ago'] = df['formatted_date'].rsub(pd.Timestamp('today')).dt.days
# df.drop(['date', 'days'], axis=1, inplace=True)
df.head()

,id,store_name,store_email,department,income,date_measured,country,formatted_date,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states,2006-02-04,6654
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,uk,2006-01-04,6685
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states,2003-09-12,7530
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,uk,2006-05-08,6561
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,uk,1973-01-21,18721
